# Gabor Thesis calculations to obtain the sensor detection

### 

In [10]:
import numpy as np
from scipy.integrate import quad

# Constants
planck_constant = 6.62607004e-34  # m^2 kg/s
speed_of_light = 299792458  # m/s
boltzmann_constant = 1.38064852e-23 #m^2 kg s-2 K-1

## Functions


In [11]:
def radiative_spectral_emittance(wavelength_, temperature_):
    """
    The radiation from a black body at a given
    wavelength and temperature is given by
    :param wavelength_: in m
    :param temperature_: in kelvin
    :return:
    """

    I_star = (2 * np.pi * planck_constant * speed_of_light ** 2) / \
             (wavelength_ ** 5 * (np.exp(planck_constant * speed_of_light / (wavelength_ * boltzmann_constant * temperature_)) - 1))
    return I_star


def radiant_flux_perratioflux_integral(lambda_interval_bottom_, lambda_interval_top_, temperature_):
    """
    Equation 2.7 from Gabor thesis
    :param lambda_interval_bottom_: integral limit
    :param lambda_interval_top_: integral limit
    :param temperature_:
    :return:
    """
    F_star_fluxratio = quad(radiative_spectral_emittance, lambda_interval_bottom_, lambda_interval_top_,
                            args=temperature_)
    return F_star_fluxratio


def radiant_flux_calculator(flux_sun_, lambda_interval_bottom_, lambda_interval_top_, temperature_):
    """
    Total power in Watts/m^2
    :param flux_sun_:
    :param lambda_interval_bottom_:
    :param lambda_interval_top_:
    :param temperature_:
    :return:
    """
    flux_star_fluxratio = radiant_flux_perratioflux_integral(lambda_interval_bottom_, lambda_interval_top_, temperature_)
    radiant_flux = np.sqrt(flux_sun_ * flux_star_fluxratio[0])
    radiant_flux_error = ((1/2)*np.sqrt(flux_sun_ * flux_star_fluxratio[0])/(flux_star_fluxratio[0]))*flux_star_fluxratio[1]
    return radiant_flux, radiant_flux_error


def planck_einstein_relation(wavelength_):
    """
    This function calculates the energy of a photon in a specific wavelength
    :param wavelength_ in meter
    :return: energy_photon in joule
    """
    energy_photon = (planck_constant * speed_of_light) / wavelength_
    return energy_photon


def photon_spectral_emittance(radiant_flux_, wavelength_):
    """
    Function for the luminosity of a star. [photons/(s*mm^2)]
    :param radiant_flux_:
    :param wavelength_:
    :return:
    """

    L_star = radiant_flux_/planck_einstein_relation(wavelength_)
    return L_star


def sensor_photon_irradiance(wavelength_, radiant_flux_, quantum_efficiency_):
    L_star = photon_spectral_emittance(radiant_flux_, wavelength_)
    E_sensor = L_star * quantum_efficiency_
    return E_sensor


def total_number_of_incident_photon_per_second_per_area(lambda_interval_bottom_, lambda_interval_top_,
                                                        radiant_flux_, quantum_efficiency_):
    E_range = quad(sensor_photon_irradiance, lambda_interval_bottom_, lambda_interval_top_,
                            args=(radiant_flux_, quantum_efficiency_))
    return E_range


def flux_of_photons_intensity(E_range_, aperture_area_):
    phi_sensor = E_range_ * aperture_area_
    return phi_sensor

def flux_intensity_scaled(phi_sensor_, magnitude_):
    phi_st = phi_sensor_ * 10**(0.4*magnitude_)
    return phi_st

def photoelectrons_per_exposure_cauculator(E_range_pe_smm2_, magnitude_star_, exposuretime_sec_, diameter_telescope_mm2_):
    photoelectrons_per_exposure = E_range_pe_smm2_ * (1.0/(2.5**(magnitude_star_ - 0)) * exposuretime_sec_ * np.pi * (diameter_telescope_mm2_/2)**2)
    return photoelectrons_per_exposure    


## Assumptions

In [40]:
# Wavelength for passband:
# Gabor example:
# lambda_interval_bottom = 400
# lambda_interval_top = 800
lambda_interval_bottom = 400
lambda_interval_top = 800

# TODO: change to K' band , 
# (center: 2120nm), (cut on: 1950nm) (cut off: 2290nm)

# temperature star
temperature = 5778  # K

# sun's flux
flux_sun = 1361  # W/m^2

# aperture telescope
diameter = 1500
aperture_area = np.pi * (diameter/2)**2

# magnitude star 
magnitude_star = 0.0

# exposure time
exposure = 1.0

radiant_flux, radiant_flux_error = radiant_flux_calculator(flux_sun_=flux_sun,
                                                            lambda_interval_bottom_=lambda_interval_bottom,
                                                            lambda_interval_top_= lambda_interval_top,
                                                            temperature_=temperature)

print('F_star [W/m^2]: ', radiant_flux)
E_range = total_number_of_incident_photon_per_second_per_area(lambda_interval_bottom_=lambda_interval_bottom*1e-9,
                                                              lambda_interval_top_=lambda_interval_top*1e-9,
                                                              radiant_flux_=radiant_flux,
                                                              quantum_efficiency_=0.45)
# print('Recheck E range units')
print('E_range [photoelectrons / s m^2]: ', E_range[0])

# phi_sensor = flux_of_photons_intensity(E_range_=E_range[0], aperture_area_=aperture_area)
# print('phi_sensor [photoelectrons / s]: ', phi_sensor)
# # Square root for error
# print("")

# TODO check units.

# phi_for_specific_star = flux_intensity_scaled(phi_sensor_=phi_sensor, magnitude_=0)
# print('phi_specific_star [photons / s]: ', phi_for_specific_star)
# print("")

photoelectrons = photoelectrons_per_exposure_cauculator(E_range[0], magnitude_star, exposure, diameter)
print('[photoelectrons / s]: ', photoelectrons)

F_star [W/m^2]:  3.0529036238072903e-08
E_range [photoelectrons / s m^2]:  16598.166804204706
[photoelectrons / s]:  29331381878.520454


123.84361237597528


## Comparing with Nick's notebook

In [52]:
# Wavelength for passband:
lambda_interval_bottom = 1300
lambda_interval_top = 1900

# temperature star
temperature = 7880  # K

# sun's flux
flux_sun = 1361  # W/m^2

# aperture telescope
diameter = 85*2 ##mm
aperture_area = np.pi * (diameter/2)**2

# magnitude star 
magnitude_star = 15.0

# exposure time
exposure = 1.0

radiant_flux, radiant_flux_error = radiant_flux_calculator(flux_sun_=flux_sun,
                                                            lambda_interval_bottom_=lambda_interval_bottom,
                                                            lambda_interval_top_= lambda_interval_top,
                                                            temperature_=temperature)

print('F_star [W/m^2]: ', radiant_flux)
E_range = total_number_of_incident_photon_per_second_per_area(lambda_interval_bottom_=lambda_interval_bottom*1e-9,
                                                              lambda_interval_top_=lambda_interval_top*1e-9,
                                                              radiant_flux_=radiant_flux,
                                                              quantum_efficiency_=0.45)
# print('Recheck E range units')
print('E_range [photoelectrons / s m^2]: ', E_range[0])

# phi_sensor = flux_of_photons_intensity(E_range_=E_range[0], aperture_area_=aperture_area)
# print('phi_sensor [photoelectrons / s]: ', phi_sensor)
# # Square root for error
# print("")

# TODO check units.

# phi_for_specific_star = flux_intensity_scaled(phi_sensor_=phi_sensor, magnitude_=0)
# print('phi_specific_star [photons / s]: ', phi_for_specific_star)
# print("")

photoelectrons = photoelectrons_per_exposure_cauculator(E_range[0], magnitude_star, exposure, diameter)
print(f'We see {photoelectrons}[photoelectrons / s] in the H filter, from a Star mag {magnitude_star} and temperature: {temperature} \n using a {diameter/2} mm radius telescope')

F_star [W/m^2]:  5.3630778503651165e-09
E_range [photoelectrons / s m^2]:  11663.29130734654
We see 284.2553955474867[photoelectrons / s] in the H filter, from a Star mag 15.0 and temperature: 7880 
 using a 85.0 mm radius telescope


## TODO:
* Check function flux_intensity_scaled(phi_sensor_, magnitude_)
* Check units for E range
* Calculates  FWC 



In [4]:
def signal_to_noise_ratio(photoelectrons_per_second_signal, dark_current_noise, read_out_noise, diffuse_background):
    """
    This function calculated the signal to noise, using as input the count of photoelectrons per second for all of these
    signal, dark_current_noise, read_out_noise, and diffuse_background.
    :param photoelectrons_per_second_signal:
    :param dark_current_noise:
    :param read_out_noise:
    :param diffuse_background:
    :return: signal to noise ratio
    """
    # This is the formula found in the paper. but it is weird
    # snr = photoelectrons_per_second_signal / np.sqrt(photoelectrons_per_second_signal + dark_current_noise +
    #                                                  read_out_noise + diffuse_background)
    # so I wll try with the square
    snr = photoelectrons_per_second_signal / np.sqrt(photoelectrons_per_second_signal**2 + dark_current_noise**2 +
                                                     read_out_noise**2 + diffuse_background**2)

    return snr